In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder


d:\ML Training using GPU\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# current_epoch = 0

# Load your dataset
data = pd.read_excel('Training Data.xlsx')  # Replace with your actual dataset path

# Clean the text data (handle missing values and ensure all are strings)
data['Text'] = data['Text'].fillna('')  # Fill NaN values with empty string
data['Text'] = data['Text'].astype(str)  # Ensure all entries are strings

# Preprocess Sentiment labels (convert them to numerical values)
label_encoder = LabelEncoder()
data['sentiment'] = label_encoder.fit_transform(data['sentiment'])

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)


---
# Start here for a new epoch 
---

In [ ]:
# Load BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained('./final')

# Tokenization function
def tokenize_function(examples):
    # Tokenize the text and make sure to include 'labels' for the target
    return tokenizer(examples['Text'], padding='max_length', truncation=True)

# Convert data to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data[['Text', 'sentiment']])
val_dataset = Dataset.from_pandas(val_data[['Text', 'sentiment']])

# Add the 'labels' column, which should be the sentiment column
train_dataset = train_dataset.map(lambda x: {'labels': x['sentiment']}, batched=True)
val_dataset = val_dataset.map(lambda x: {'labels': x['sentiment']}, batched=True)

# Apply tokenization to both train and validation datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map: 100%|██████████| 11269/11269 [00:03<00:00, 3143.26 examples/s]


In [14]:

# Load pre-trained BERT model
# model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=len(label_encoder.classes_))
model = BertForSequenceClassification.from_pretrained('./final', num_labels=len(label_encoder.classes_))


In [11]:

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',              # Output directory
    num_train_epochs=1,                  # Number of epochs
    per_device_train_batch_size=8,       # Batch size for training
    per_device_eval_batch_size=16,       # Batch size for evaluation
    warmup_steps=500,                    # Warmup steps for learning rate scheduler
    weight_decay=0.01,                   # Strength of weight decay
    logging_dir='./logs',                # Directory for storing logs
    logging_steps=10,                    # Log every 10 steps
    evaluation_strategy='epoch',         # Evaluate every epoch
    save_strategy='epoch',               # Save model every epoch
)


d:\ML Training using GPU\myenv\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)  # Convert logits to predicted class
    return {"accuracy": accuracy_score(labels, predictions)}

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train(resume_from_checkpoint=True)


C:\Users\param\AppData\Local\Temp\ipykernel_12404\1159032862.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=5635, training_loss=0.0, metrics={'train_runtime': 0.002, 'train_samples_per_second': 22535946.894, 'train_steps_per_second': 2817368.344, 'total_flos': 1.1859893634797568e+16, 'train_loss': 0.0, 'epoch': 1.0})

In [16]:

current_epoch = trainer.state.epoch
print(f"Epoch {current_epoch}")

Epoch 1.0


In [ ]:

# Train the model0
trainer.train()

trainer.save_model('./final')
tokenizer.save_pretrained('./final')
trainer.save_state()

# print(f"\nEpoch {epoch + 1} completed and model saved at {checkpoint_path}\n")


Epoch,Training Loss,Validation Loss
1,0.551700,0.634172


NameError: name 'epoch' is not defined

In [21]:
# Evaluate the model
results = trainer.evaluate()


NameError: name 'np' is not defined

In [ ]:

# Print the evaluation results, which include accuracy
print(f"Accuracy: {results['eval_accuracy']}")

KeyError: 'eval_accuracy'